#### LeetCode - SQL - #185

Refer [185. Department Top Three Salaries](https://leetcode.com/problems/department-top-three-salaries/description/)

A company's executives are interested in seeing who earns the most money in each of the company's departments. A  **high earner**  in a department is an employee who has a salary in the  **top three unique**  salaries for that department.

Write a solution to find the employees who are  **high earners**  in each of the departments.

Return the result table  **in any order**.

In [0]:
employee_data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee_columns = ['id', 'name', 'salary', 'departmentId']
employee_schema = {'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'}

department_data = [[1, 'IT'], [2, 'Sales']]
department_columns = ['id', 'name']
department_schema = {'id':'Int64', 'name':'object'}

In [0]:
import pandas

employee_pandas_dataframe = pandas.DataFrame(data=employee_data, columns=employee_columns).astype(employee_schema)
department_pandas_dataframe = pandas.DataFrame(data=department_data, columns=department_columns).astype(department_schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
employee_dataframe = spark_context.createDataFrame(employee_pandas_dataframe)
employee_dataframe.printSchema()
employee_dataframe.show(5)

department_dataframe = spark_context.createDataFrame(department_pandas_dataframe)
department_dataframe.printSchema()
department_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- departmentId: long (nullable = true)

+---+-----+------+------------+
| id| name|salary|departmentId|
+---+-----+------+------------+
|  1|  Joe| 85000|           1|
|  2|Henry| 80000|           2|
|  3|  Sam| 60000|           2|
|  4|  Max| 90000|           1|
|  5|Janet| 69000|           1|
+---+-----+------+------------+
only showing top 5 rows

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)

+---+-----+
| id| name|
+---+-----+
|  1|   IT|
|  2|Sales|
+---+-----+



In [0]:
joined_dataframe \
    = employee_dataframe \
        .join(department_dataframe, employee_dataframe.departmentId == department_dataframe.id, "inner") \
        .select(employee_dataframe.name, employee_dataframe.salary, department_dataframe.name.alias("department_name"))
joined_dataframe.show(5)

+-----+------+---------------+
| name|salary|department_name|
+-----+------+---------------+
|  Joe| 85000|             IT|
|Henry| 80000|          Sales|
|  Sam| 60000|          Sales|
|  Max| 90000|             IT|
|Janet| 69000|             IT|
+-----+------+---------------+
only showing top 5 rows



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

window_spec = Window.partitionBy(joined_dataframe.department_name).orderBy(joined_dataframe.salary.desc())

joined_dataframe = joined_dataframe.withColumn("department_salary_rank", dense_rank().over(window_spec))
joined_dataframe.show()

joined_dataframe \
    .filter(joined_dataframe.department_salary_rank <= 3) \
    .select(joined_dataframe.department_name.alias("Department"), joined_dataframe.name.alias("Employee"), joined_dataframe.salary.alias("Salary")) \
    .show()

+-----+------+---------------+----------------------+
| name|salary|department_name|department_salary_rank|
+-----+------+---------------+----------------------+
|  Max| 90000|             IT|                     1|
|  Joe| 85000|             IT|                     2|
|Randy| 85000|             IT|                     2|
| Will| 70000|             IT|                     3|
|Janet| 69000|             IT|                     4|
|Henry| 80000|          Sales|                     1|
|  Sam| 60000|          Sales|                     2|
+-----+------+---------------+----------------------+

+----------+--------+------+
|Department|Employee|Salary|
+----------+--------+------+
|        IT|     Max| 90000|
|        IT|     Joe| 85000|
|        IT|   Randy| 85000|
|        IT|    Will| 70000|
|     Sales|   Henry| 80000|
|     Sales|     Sam| 60000|
+----------+--------+------+

